<a href="https://colab.research.google.com/github/the-SQuAD-squad/IR-QA/blob/main/QA_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Init { form-width: "25%" }
import os
import random
import math
import numpy as np
import tensorflow as tf
import json
import pandas as pd

pd.set_option('display.max_colwidth', -1)

# fix random seeds
seed_value = 42 #@param {type:"integer"}

os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

tf.compat.v1.set_random_seed(seed_value)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # Remove the CWD from sys.path while we load stuff.


In [2]:
#@title df creation { form-width: "25%" }

# dataset is copyed to public git repo for fast access within colab 
!wget 'https://raw.githubusercontent.com/the-SQuAD-squad/data/main/SQuAD/squad1.1.zip'
!unzip -o squad1.1.zip

with open("training_set.json", "r") as f:
    json_file = json.load(f)
data = json_file["data"]

rows = []
for document in data:
  for par in document['paragraphs']:
    for qas in par['qas']:
      rows.append({
        'id' : qas['id'],
        'title': document["title"],
        'passage': par['context'],
        'question' : qas['question'],
        'answer_idx' : (qas['answers'][0]['answer_start'], 
                    qas['answers'][0]['answer_start'] + len(qas['answers'][0]['text'])),
        'answer_text' : qas['answers'][0]['text']
      })

df_original = pd.DataFrame(rows)

--2021-02-08 16:11:13--  https://raw.githubusercontent.com/the-SQuAD-squad/data/main/SQuAD/squad1.1.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7638822 (7.3M) [application/zip]
Saving to: ‘squad1.1.zip’

squad1.1.zip        100%[===================>]   7.28M  27.8MB/s    in 0.3s    

2021-02-08 16:11:14 (27.8 MB/s) - ‘squad1.1.zip’ saved [7638822/7638822]

Archive:  squad1.1.zip
  inflating: training_set.json       


In [ ]:
df_original

,id,title,passage,question,answer_idx,answer_text
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?,"(515, 541)",Saint Bernadette Soubirous
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",What is in front of the Notre Dame Main Building?,"(188, 213)",a copper statue of Christ
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",The Basilica of the Sacred heart at Notre Dame is beside to which structure?,"(279, 296)",the Main Building
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",What is the Grotto at Notre Dame?,"(381, 420)",a Marian place of prayer and reflection
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mar

In [ ]:
import nltk
import re 
import math

def preprocess_text(text):
    """
    Given an iterable containing sentences, pre-process each sentence.

    :param: 
        - text: list of text to be pre-processed (Iterable)
    :return:
        - text: pre-processed text (List)
    """

    REPLACE_WITH_SPACE = re.compile(r"\n") 
    text = [REPLACE_WITH_SPACE.sub(" ", line) for line in text]

    # we don't remove symbols, but just put a space before and after them. We did this because we noticed that Glove contains an embedding also for
    # them, so, in this way, we are able to split these symbols from the text when computing sentence tokens
    text = [re.sub(r"([(.;:!\'ˈ~?,\"(\[\])\\\/\-–\t```<>_#$€@%*+—°′″“”×’^₤₹‘])", r' \1 ', line) for line in text]

    # we noticed that in the text sometimes we find numbers and the following word merged together (ex: 1980february),
    # so we put a space between the number and the word
    text = [re.sub(r"(\d+)([a-z]+)", r'\1 \2', line) for line in text] 
    text = [re.sub('\s{2,}', ' ', line.strip()) for line in text]   # replacing more than one consecutive blank spaces with only one of them

    return text


# Creating a copy of the original dataframe (we do this because we want to be able to compare the results of our processing with the original data)
df = df_original.copy()

# pre-process passage and question text
df['passage'] = preprocess_text(df_original['passage'])
df['question'] = preprocess_text(df_original['question'])
df['answer_text'] = preprocess_text(df_original['answer_text'])

In [ ]:
import random as rand
# Comparing Original and Pre-Processed
for i in range(3):
    a = rand.randint(0,1000)
    print('ORIGINAL AND PREPROCESSED PASSAGE:')
    print(df_original.iloc[a]['passage'])
    print(df.iloc[a]['passage'])
    
    print()
    print('ORIGINAL AND PREPROCESSED QUESTION:')
    print(df_original.iloc[a]['question'])
    print(df.iloc[a]['question'])
    print()


In [ ]:
unwanted_rows = set()   # this set will contain the indices of rows containing errors (thus we will remove these rows from the dataframe)
unwanted_id = set()    # this set will contain the IDs of rows containing errors 
word_idx = []   # will contain the start and end indices of each answer in the corresponing passage
impossible_count = 0

for i in df_original.index:

    # extracting one answer and the corresponding passage
    answer = np.array(df["answer_text"][i].split())
    passage = np.array(df["passage"][i].split())

    l = len(answer)
    idx = []    # this list will contain the start and end indices of the answer occurrence in the corresponding passage (these could be more than 1! ex: we find the answer "rome" in 2 distinct positions in the same passage)
    counts = []     # counts will contain how many characters there are before the answer start in the passage
    char_count = 0
    for j in range(len(passage)-l+1):   # for each token in the passage, check if the answer starts from that token
        if (answer == passage[j:j+l]).all():
            idx.append((j, j+l))    # j is the start_index and j+l is the end_index of the answer in the passage
            counts.append(char_count)    # char_count is the count of characters before the start of the answer
        char_count += len(passage[j])
    if len(counts) == 0:    # no answer found in the passage. Probably there is an error in the dataset (for instance the answer is "7", but in the text it is written like "seven")
        unwanted_rows.add(i)
        unwanted_id.add(df_original["id"][i])
        word_idx.append((-1, -1))   # stating that there was an error
        impossible_count+=1

        # printing original question, answer and passage (by using the answer id to find the right dataframe row)
        print(str(df_original["question"][df_original["id"] == df["id"][i]]))
        print(str(df_original['passage'][df_original['id'] == df['id'][i]]))
        print(str(df_original['answer_text'][df_original['id'] == df['id'][i]]))
        print("answer extracted from: {0}".format((df_original['passage'][i][df_original["answer_idx"][i][0]-3:df_original["answer_idx"][i][1]+3])))
        print()

    else:   # answer found in the passage
        # if more than one answer correspondence was found in the passage, we take the one whose start index is nearer the start index given in the dataset 
        # (these 2 starting indices do not match perfectly because we are working on the preoprocessed text, so our starting index is a little bit different from the original)
        n_spaces_original = df_original["passage"][i][:df_original["answer_idx"][i][0]].count(" ")    # counting how many spaces there are in the original passage before the answer
        n_newline_original = df_original["passage"][i][:df_original["answer_idx"][i][0]].count("\n")    # counting how many newline characters there are in the original passage before the answer
        s = np.abs(np.array(counts)-(df["answer_idx"][i][0]-n_spaces_original-n_newline_original))

        if (0 not in s) and len(s)>1:   # in this case the answer was found in the passage, but the start index specified in the dataset is not the right one (it indicates a wrong occurrence of the answer)
            unwanted_rows.add(i)
            unwanted_id.add(df_original["id"][i])
            word_idx.append((-1, -1))   # stating that there was an error
            impossible_count+=1

            print(str(df_original["question"][df_original["id"]==df["id"][i]]))
            print(str(df_original['passage'][df_original['id'] == df['id'][i]]))
            print(str(df_original['answer_text'][df_original['id'] == df['id'][i]]))
            print("answer extracted from: {0}".format((df_original['passage'][i][df_original["answer_idx"][i][0]-3:df_original["answer_idx"][i][1]+3])))
            print()
        else:
            word_idx.append(idx[np.argmin(s)])
            # print(df_original["answer_text"][i])
            # print(passage[idx[np.argmin(s)][0]:idx[np.argmin(s)][1]])

print("The number of rows that we will remove from the dataframe (because contain errors) are {0}".format(impossible_count))

In [ ]:
# adding a new column to the dataframe containing the word indices of the answer in the splitted passage
df["word_idx_answer"] = word_idx
df["passage"]=df["passage"].str.lower()
df["question"]=df["question"].str.lower()
df["answer_text"]=df["answer_text"].str.lower()
df

In [ ]:
import numpy as np

def build_vocabulary(text):
    """
    Given a list of words, builds the corresponding word vocabulary and the mappings from words to indices and vice-versa.

    :param: 
        - text: list of words from which we want to build the vocabularies (List)
    :return:
        - idx2word: index to word mapping (Dict)
        - word2idx: word to index mapping (Dict)
        - set_vocab: set of unique terms that build up the vocabulary
    """

    # Creating a set to eliminate repeated words
    set_vocab = ['<PAD>']+sorted(set(text)) # here we add the padding token as the first element of the set

    # Creating a mapping from unique words to indices
    word2idx = {u:i for i, u in enumerate(set_vocab)}   # the padding token will have 0 index
    # Creating a mapping from indices to unique words
    idx2word = {i:u for i, u in enumerate(set_vocab)}

    return idx2word,word2idx,set_vocab

# Creating a list containing all the passage and question text splitted in words
text =  ' '.join(np.concatenate((df['passage'],df['question']))).split(' ')
# Displaying first 100 words
print(text[:100])
# calling the build_vocabulary function to obtain the vocab and the mappings
idx_to_word, word_to_idx, word_listing = build_vocabulary(text)

print('[Debug] Index -> Word vocabulary size: {}'.format(len(idx_to_word)))
print('[Debug] Word -> Index vocabulary size: {}'.format(len(word_to_idx)))
print('[Debug] Some words: {}'.format([(idx_to_word[idx], idx) for idx in np.arange(100)]))

['architecturally', ',', 'the', 'school', 'has', 'a', 'catholic', 'character', '.', 'atop', 'the', 'main', 'building', "'", 's', 'gold', 'dome', 'is', 'a', 'golden', 'statue', 'of', 'the', 'virgin', 'mary', '.', 'immediately', 'in', 'front', 'of', 'the', 'main', 'building', 'and', 'facing', 'it', ',', 'is', 'a', 'copper', 'statue', 'of', 'christ', 'with', 'arms', 'upraised', 'with', 'the', 'legend', '"', 'venite', 'ad', 'me', 'omnes', '"', '.', 'next', 'to', 'the', 'main', 'building', 'is', 'the', 'basilica', 'of', 'the', 'sacred', 'heart', '.', 'immediately', 'behind', 'the', 'basilica', 'is', 'the', 'grotto', ',', 'a', 'marian', 'place', 'of', 'prayer', 'and', 'reflection', '.', 'it', 'is', 'a', 'replica', 'of', 'the', 'grotto', 'at', 'lourdes', ',', 'france', 'where', 'the', 'virgin', 'mary']
[Debug] Index -> Word vocabulary size: 82630
[Debug] Word -> Index vocabulary size: 82630
[Debug] Some words: [('<PAD>', 0), ('!', 1), ('"', 2), ('#', 3), ('$', 4), ('%', 5), ('&', 6), ("'", 7)

In [ ]:
import gensim
import gensim.downloader as gloader

def load_embedding_model(model_type, embedding_dimension=50):
    """
    Loads a pre-trained word embedding model via gensim library.

    :params:
        - model_type: name of the word embedding model to load.
        - embedding_dimension: size of the embedding space to consider

    :return:
        - pre-trained word embedding model (gensim KeyedVectors object)
    """

    download_path = ""

    # Find the correct embedding model name
    if model_type.strip().lower() == 'word2vec':
        download_path = "word2vec-google-news-300"

    elif model_type.strip().lower() == 'glove':
        download_path = "glove-wiki-gigaword-{}".format(embedding_dimension)

    else:
        raise AttributeError("Unsupported embedding model type! Available ones: word2vec, glove")

    # Check download
    try:
        emb_model = gloader.load(download_path)
    except ValueError as e:
        print("Invalid embedding model name! Check the embedding dimension:")
        print("Word2Vec: 300")
        print("Glove: 50, 100, 200, 300")
        raise e

    return emb_model


def check_OOV_terms(embedding_model, word_listing):
    """
    Checks differences between pre-trained embedding model vocabulary
    and dataset specific vocabulary in order to highlight out-of-vocabulary terms.

    :params:
        - embedding_model: pre-trained word embedding model (gensim wrapper)
        - word_listing: dataset specific vocabulary (list)

    :return:
        - list of OOV terms
    """
    # Creating a list for the OOV words
    oov = []
    for word in word_listing:
        # Checking if the word is in the embedding_model
        if word not in embedding_model:
            oov.append(word)
    return oov


def build_embedding_matrix_w_random(embedding_model, embedding_dimension, word_to_idx, oov_terms):
    """
    Builds the embedding matrix of a specific dataset given a pre-trained word embedding model

    :params: 
        - embedding_model: pre-trained word embedding model (gensim wrapper)
        - word_to_idx: vocabulary map (word -> index) (dict)
        - oov_terms: list of OOV terms (list)

    :return
        - embedding matrix that assigns a high dimensional vector to each word in the dataset specific vocabulary (shape |V| x d)
    """
    embedding_matrix = []
    for word in word_to_idx:
        if word in oov_terms:
            embedding_matrix.append(np.random.rand(embedding_dimension))
        else:
             embedding_matrix.append(embedding_model[word])
    return np.array(embedding_matrix)

# we used Glove with embedding dimension 100 for our final tests
embedding_model_type = "Glove"
embedding_dimension = 100
embedding_model = load_embedding_model(embedding_model_type, embedding_dimension)

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
# checking how many OOV terms we have
oov_terms = check_OOV_terms(embedding_model, word_listing)

print("Total OOV terms: {0} ({1:.2f}%)".format(len(oov_terms), len(oov_terms)/len(word_listing)*100))

embedding_matrix = build_embedding_matrix_w_random(embedding_model, embedding_dimension, word_to_idx, oov_terms)

print("Embedding matrix shape: {}".format(embedding_matrix.shape))

Total OOV terms: 15985 (19.35%)
Embedding matrix shape: (82630, 100)


In [ ]:
print(oov_terms)   # this was useful to understand if we could improve pre-processing

['<PAD>', '0000222556', '000a', '000rmb', '0018', '0028', '0029670', '0042', '0043', '0054', '0065', '0071', '0079', '00794', '00b7', '00e9', '012f', '0243', '0307', '0358', '036', '042', '057', '058', '05946', '062', '064', '066', '067', '069', '073', '078', '079', '083', '085', '088', '096', '097', '0fm', '100l', '10217', '10925', '10ffff', '10z', '10¢', '10−10', '10−12', '10−19', '10−2', '10−3', '10−34', '10−6', '10−8', '10−9', '1101010', '11092', '11114', '11172', '11246', '12232', '12291', '122ad', '1234567', '12750', '130h', '130j', '131\u202f000', '133cs', '13500', '13526', '13818', '145a', '146a', '14fdr', '14−17', '15408', '16041', '16384', '177847', '178b', '17½', '17\u202f000', '180ad', '18578', '1881−82', '1911a1', '1967a', '1967b', '197bc', '19n', '19sc111', '19˚n', '19˚s', '1fm', '1n4148', '1q08', '1\u202f800', '1⁄10', '1⁄14', '1⁄2', '1⁄3', '1⁄4', '1⁄8', '20011', '20012', '20015', '2002…', '2013−14', '20147', '2053b', '20740', '20gb', '20\u202f000', '21⁄32', '22300', '230

In [ ]:
for i,question in enumerate(df['question']):
    if len(question)<=10:
        print(df_original.iloc[i]) 
        print(df_original['passage'][i])
        print()
        unwanted_rows.add(i)
        unwanted_id.add(df["id"][i])

In [ ]:
# ERROR CHECK before removing rows
print("Number of errors found: {0}\n".format(str(len(unwanted_id))))
some_error_id = random.sample(unwanted_id, 5)
for id in some_error_id:
    row_with_error = df_original[df_original["id"] == id].to_dict("list")
    print("Question: {0}".format(row_with_error["question"][0]))
    print("Passage: {0}".format(row_with_error["passage"][0]))
    print("Answer: {0}".format(row_with_error["answer_text"][0]))
    print("Answer extracted from: {0}".format(row_with_error["passage"][0][row_with_error["answer_idx"][0][0]-3:row_with_error["answer_idx"][0][1]+3]))
    print()

In [ ]:
# Creating a txt file containing IDs of rows with errors for tutors
with open("error IDs.txt", "a") as f:
    for error_id in unwanted_id:
        f.write(error_id + "\n")

In [ ]:
df

,id,title,passage,question,answer_idx,answer_text,word_idx_answer
0,5733be284776f41900661182,University_of_Notre_Dame,"architecturally , the school has a catholic character . atop the main building ' s gold dome is a golden statue of the virgin mary . immediately in front of the main building and facing it , is a copper statue of christ with arms upraised with the legend "" venite ad me omnes "" . next to the main building is the basilica of the sacred heart . immediately behind the basilica is the grotto , a marian place of prayer and reflection . it is a replica of the grotto at lourdes , france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858 . at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ) , is a simple , modern stone statue of mary .",to whom did the virgin mary allegedly appear in 1858 in lourdes france ?,"(515, 541)",saint bernadette soubirous,"(103, 106)"
1,5733be284776f4190066117f,University_of_Notre_Dame,"architecturally , the school has a catholic character . atop the main building ' s gold dome is a golden statue of the virgin mary . immediately in front of the main building and facing it , is a copper statue of christ with arms upraised with the legend "" venite ad me omnes "" . next to the main building is the basilica of the sacred heart . immediately behind the basilica is the grotto , a marian place of prayer and reflection . it is a replica of the grotto at lourdes , france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858 . at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ) , is a simple , modern stone statue of mary .",what is in front of the notre dame main building ?,"(188, 213)",a copper statue of christ,"(38, 43)"
2,5733be284776f41900661180,University_of_Notre_Dame,"architecturally , the school has a catholic character . atop the main building ' s gold dome is a golden statue of the virgin mary . immediately in front of the main building and facing it , is a copper statue of christ with arms upraised with the legend "" venite ad me omnes "" . next to the main building is the basilica of the sacred heart . immediately behind the basilica is the grotto , a marian place of prayer and reflection . it is a replica of the grotto at lourdes , france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858 . at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ) , is a simple , modern stone statue of mary .",the basilica of the sacred heart at notre dame is beside to which structure ?,"(279, 296)",the main building,"(58, 61)"
3,5733be284776f41900661181,University_of_Notre_Dame,"architecturally , the school has a catholic character . atop the main building ' s gold dome is a golden statue of the virgin mary . immediately in front of the main building and facing it , is a copper statue of christ with arms upraised with the legend "" venite ad me omnes "" . next to the main building is the basilica of the sacred heart . immediately behind the basilica is the grotto , a marian place of prayer and reflection . it is a replica of the grotto at lourdes , france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858 . at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ) , is a simple , modern stone statue of mary .",what is the grotto at notre dame ?,"(381, 420)",a marian place of prayer and reflection,"(77, 84)"
4,5733be284776f4190066117e,University_of_Notre_Dame,"architecturally , the school has a catholic character . atop the main building ' s gold dome is a golden statue of the virgin mary . immediately in front of the main building and facing it , is a copper statue of christ with arms upraised with the legend "" venite ad me omnes "" . next to the main building is the basilica of the sacred heart . immed

In [ ]:
df_clean = df.drop(list(unwanted_rows))
df_clean = df_clean.reset_index()
df_clean.to_pickle("df_clean.pkl")
!zip df_clean.pkl.zip df_clean.pkl
df_clean

  adding: df_clean.pkl (deflated 88%)


,index,id,title,passage,question,answer_idx,answer_text,word_idx_answer
0,0,5733be284776f41900661182,University_of_Notre_Dame,"architecturally , the school has a catholic character . atop the main building ' s gold dome is a golden statue of the virgin mary . immediately in front of the main building and facing it , is a copper statue of christ with arms upraised with the legend "" venite ad me omnes "" . next to the main building is the basilica of the sacred heart . immediately behind the basilica is the grotto , a marian place of prayer and reflection . it is a replica of the grotto at lourdes , france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858 . at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ) , is a simple , modern stone statue of mary .",to whom did the virgin mary allegedly appear in 1858 in lourdes france ?,"(515, 541)",saint bernadette soubirous,"(103, 106)"
1,1,5733be284776f4190066117f,University_of_Notre_Dame,"architecturally , the school has a catholic character . atop the main building ' s gold dome is a golden statue of the virgin mary . immediately in front of the main building and facing it , is a copper statue of christ with arms upraised with the legend "" venite ad me omnes "" . next to the main building is the basilica of the sacred heart . immediately behind the basilica is the grotto , a marian place of prayer and reflection . it is a replica of the grotto at lourdes , france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858 . at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ) , is a simple , modern stone statue of mary .",what is in front of the notre dame main building ?,"(188, 213)",a copper statue of christ,"(38, 43)"
2,2,5733be284776f41900661180,University_of_Notre_Dame,"architecturally , the school has a catholic character . atop the main building ' s gold dome is a golden statue of the virgin mary . immediately in front of the main building and facing it , is a copper statue of christ with arms upraised with the legend "" venite ad me omnes "" . next to the main building is the basilica of the sacred heart . immediately behind the basilica is the grotto , a marian place of prayer and reflection . it is a replica of the grotto at lourdes , france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858 . at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ) , is a simple , modern stone statue of mary .",the basilica of the sacred heart at notre dame is beside to which structure ?,"(279, 296)",the main building,"(58, 61)"
3,3,5733be284776f41900661181,University_of_Notre_Dame,"architecturally , the school has a catholic character . atop the main building ' s gold dome is a golden statue of the virgin mary . immediately in front of the main building and facing it , is a copper statue of christ with arms upraised with the legend "" venite ad me omnes "" . next to the main building is the basilica of the sacred heart . immediately behind the basilica is the grotto , a marian place of prayer and reflection . it is a replica of the grotto at lourdes , france where the virgin mary reputedly appeared to saint bernadette soubirous in 1858 . at the end of the main drive ( and in a direct line that connects through 3 statues and the gold dome ) , is a simple , modern stone statue of mary .",what is the grotto at notre dame ?,"(381, 420)",a marian place of prayer and reflection,"(77, 84)"
4,4,5733be284776f4190066117e,University_of_Notre_Dame,"architecturally , the school has a catholic character . atop the main building ' s gold dome is a golden statue of the virgin mary . immediately in front of the main building and facing it , is a copper statue of christ with arms upraised with the legend "" venite ad me omnes "" . next to the main building is the basilica of the sacr

In [ ]:
# PADDING
# all sequences in train and val sets will be padded with a number of tokens equal to the maximum sentence length 
MAX_LENGTH_PASSAGE = len(max(df_clean['passage'], key=len))   
passages = [[word_to_idx[el] for el in sentence.split()] for sentence in df_clean['passage']]  # passages extraction
passages_pad = tf.keras.preprocessing.sequence.pad_sequences(passages, maxlen=MAX_LENGTH_PASSAGE, padding='post') # padding passages

MAX_LENGTH_QUESTION = len(max(df_clean['question'], key=len))   
questions = [[word_to_idx[el] for el in sentence.split()] for sentence in df_clean['question']]  # questions extraction
questions_pad = tf.keras.preprocessing.sequence.pad_sequences(questions, maxlen=MAX_LENGTH_QUESTION, padding='post') # padding questions

df_clean['passage_pad'] = list(passages_pad)
df_clean['question_pad'] = list(questions_pad)

In [ ]:
#@title Skip preprocessing
import pickle
load= True #@param {type: "boolean"}

if load:
    import os
    import random
    import math
    import numpy as np
    import tensorflow as tf
    import json
    import pandas as pd

    pd.set_option('display.max_colwidth', -1)

    # fix random seeds
    seed_value = 42 #@param {type:"integer"}
    os.environ['PYTHONHASHSEED']=str(seed_value)
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.compat.v1.set_random_seed(seed_value)
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)


    !gcloud config set project feisty-mechanic-221914
    !gsutil cp gs://squad_squad/df_clean.pkl ./df_clean.pkl
    !gsutil cp gs://squad_squad/embedding_matrix.pkl ./embedding_matrix.pkl
    !gsutil cp gs://squad_squad/idx_to_word.pkl ./idx_to_word.pkl

    df_clean = pd.read_pickle("df_clean.pkl")
    with open('embedding_matrix.pkl', 'rb') as handle:
        embedding_matrix = pickle.load(handle)
    with open('idx_to_word.pkl', 'rb') as handle:
        idx_to_word = pickle.load(handle)
else:
    df_clean.to_pickle("df_clean.pkl")
    with open('embedding_matrix.pkl', 'wb') as handle:
        pickle.dump(embedding_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open('idx_to_word.pkl', 'wb') as handle:
        pickle.dump(idx_to_word, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project feisty-mechanic-221914
    !gsutil cp ./df_clean.pkl gs://squad_squad/df_clean.pkl
    !gsutil cp ./embedding_matrix.pkl gs://squad_squad/embedding_matrix.pkl
    !gsutil cp ./idx_to_word.pkl gs://squad_squad/idx_to_word.pkl

!nvidia-smi

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


Updated property [core/project].
Copying gs://squad_squad/df_clean.pkl...
- [1 files][  1.4 GiB/  1.4 GiB]   55.6 MiB/s                                   
Operation completed over 1 objects/1.4 GiB.                                      
Copying gs://squad_squad/embedding_matrix.pkl...
| [1 files][ 63.0 MiB/ 63.0 MiB]                                                
Operation completed over 1 objects/63.0 MiB.                                     
Copying gs://squad_squad/idx_to_word.pkl...
- [1 files][  1.1 MiB/  1.1 MiB]                                                
Operation completed over 1 objects/1.1 MiB.                                      
Mon Feb  8 12:16:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr

In [ ]:
#@title split { form-width: "25%" }

split_value = 0.1 #@param {type:"number"} 
val_dim = int(len(df_clean['title'].unique()) * split_value)
val_titles = np.random.choice(df_clean['title'].unique(), size=val_dim, replace=False)
passage_length = len(df_clean['passage_pad'][0])
question_length = len(df_clean['question_pad'][0])

In [ ]:
# creating train and val sets
df_val = df_clean[df_clean['title'].isin(val_titles)]
df_train = df_clean[~(df_clean['title'].isin(val_titles))]

In [ ]:
def build_model():
    input_size=len(idx_to_word)
    embedding_dim=100
    encoding_units = 256
    dropout = 0

    input_passage = tf.keras.layers.Input(shape=[None])
    input_question = tf.keras.layers.Input(shape=[None])

    # EMBEDDING
    embedding = tf.keras.layers.Embedding(input_size,
                                        embedding_dim,  
                                        weights=[embedding_matrix],
                                        trainable=False,
                                        mask_zero=True
                                        )   # trainable param is False because we use pre-trained Glove embeddings, mask_zero param is True because we have padding
    embedding_passage = embedding(input_passage)
    embedding_question = embedding(input_question)


    # ENCODING passage AND question
    encoding_passage = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(encoding_units, 
                                kernel_initializer='glorot_uniform',
                                recurrent_initializer='orthogonal',
                                dropout=dropout,
                                stateful=False,
                                return_sequences=True))(embedding_passage)   

    encoding_question = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(encoding_units, 
                                kernel_initializer='glorot_uniform',
                                recurrent_initializer='orthogonal',
                                dropout=dropout,
                                stateful=False,
                                return_sequences=True))(embedding_question)

    # ATTENTION LAYER
    #encoded_question = tf.keras.layers.RepeatVector(MAX_LENGTH_PASSAGE)(encoding_question)
    query_value_attention_seq = tf.keras.layers.Attention()([encoding_passage, encoding_question])   # out shape: [batch_size, passage_len, encoding_dim]
    # reducing over the sequence axis to produce encodings of shape [batch_size, encoding_dimension]
    #query_value_attention_seq = tf.keras.layers.GlobalAveragePooling1D()(query_value_attention_seq)
    # concatenating passage and question encodings on the sequence length dimension
    #encoded_pair = tf.keras.layers.concatenate([encoding_passage, encoding_question],axis=1)
    # reducing over the sequence axis to produce encodings of shape [batch_size, encoding_dimension]
    #encoded_pair = tf.keras.layers.GlobalAveragePooling1D()(encoded_pair)   
    # concatenating passage and question encoding pair with the attention result
    #combined = tf.keras.layers.concatenate([encoded_pair, query_value_attention_seq])

    combined = tf.keras.layers.concatenate([encoding_passage, query_value_attention_seq])  # tried add but doesnt work

    lstm_start = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(encoding_units, 
                                kernel_initializer='glorot_uniform',
                                recurrent_initializer='orthogonal',
                                dropout=dropout,
                                stateful=False,
                                return_sequences=True))(combined)


    output_start = tf.keras.layers.Dense(1)(lstm_start)
    logits = tf.squeeze(output_start, axis=[2]) # shape (batch_size, seq_len)
    output_start=tf.keras.layers.Softmax(name="answ_start")(logits)

    # output_end= tf.keras.layers.Dense(256)(combined) # this last dense layer outputs the positive class probability
    # output_end = tf.keras.layers.Dense(1)(output_end)
    # logits = tf.squeeze(output_end, axis=[2]) # shape (batch_size, seq_len)
    # output_end=tf.keras.layers.Softmax(name="answ_end")(logits)

    # output_start = tf.keras.layers.Dense(1)(query_value_attention_seq) # this last dense layer outputs the positive class probability
    # output_start = tf.keras.layers.Reshape((passage_length,))(output_start)
    # output_start=tf.keras.layers.Softmax(name="answ_start")(output_start)



    #output_start_reshaped = tf.keras.layers.Reshape((passage_length,1))(output_start)

    #combined_with_start_probabilities = tf.keras.layers.concatenate([combined, output_start_reshaped])

    lstm_end = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(encoding_units, 
                                kernel_initializer='glorot_uniform',
                                recurrent_initializer='orthogonal',
                                dropout=dropout,
                                stateful=False,
                                return_sequences=True))(combined)

    output_end = tf.keras.layers.Dense(1)(lstm_end)
    logits = tf.squeeze(output_end, axis=[2]) # shape (batch_size, seq_len)
    output_end=tf.keras.layers.Softmax(name="answ_end")(logits)


    # output_end = tf.keras.layers.Dense(1)(lstm_end) # this last dense layer outputs the positive class probability
    # output_end = tf.keras.layers.Reshape((passage_length,))(output_end)
    # output_end=tf.keras.layers.Softmax(name="answ_end")(output_end)

    # out = tf.stack([output_start, output_end],axis=-1)

    # model = tf.keras.Model([input_passage,input_question], outputs=[out]) 

    out = tf.stack([output_start, output_end],axis=-1)
    model = tf.keras.Model([input_passage,input_question], outputs=[out])  

    model.summary()
    return model

In [ ]:
def prec(y_true, y_pred):
    sampled = tf.argmax(y_pred, axis=-1)
    return 1 - tf.math.count_nonzero(tf.cast(y_true, tf.int64) - sampled) / tf.cast(len(sampled), tf.int64)

def precision_start(y_true, y_pred):
    return prec(y_true[:,0], y_pred[:,:,0])

def precision_end(y_true, y_pred):
    return prec(y_true[:,1], y_pred[:,:,1])

def dist(y_true, y_pred):
    sampled = tf.argmax(y_pred, axis=-1)
    return tf.reduce_sum(tf.abs(tf.cast(y_true, tf.int64) - sampled)) / tf.cast(len(sampled), tf.int64)

def mean_abs_dist_start(y_true_tuple, y_pred_tuple):
    return dist(y_true_tuple[:,0], y_pred_tuple[:,:,0])
    
def mean_abs_dist_end(y_true_tuple, y_pred_tuple):
    return dist(y_true_tuple[:,1], y_pred_tuple[:,:,1])

In [ ]:
#@title train {form-width: "30%"}
cc = tf.keras.losses.SparseCategoricalCrossentropy()
α = 1
β = 0
def custom_loss(y_true, y_pred):
    sampled_start = tf.argmax(y_pred[:,:,0], axis=-1)
    sampled_end = tf.argmax(y_pred[:,:,1], axis=-1)
    precedence_constraint = - tf.math.minimum(sampled_end - sampled_start, 
                                             tf.zeros(len(sampled_end), dtype=tf.int64)) / tf.cast(len(sampled_end), dtype=tf.int64)
    precedence_constraint = tf.cast(precedence_constraint, tf.float32)

    start_loss = cc(y_true[:,0], y_pred[:,:,0])
    end_loss = cc(y_true[:,1], y_pred[:,:,1])
    cross_entropy = start_loss + end_loss 

    return α*cross_entropy + β*precedence_constraint

batch_size = 16
epochs = 1000

ENABLE_WANDB = False        #@param {type:"boolean"}
wandb_experiment_name = "still_plaing_with_layer_norm"  #@param {type: "string"}
if ENABLE_WANDB:
    !pip install wandb > /dev/null
    !wandb login wandb_api_token
    import wandb
    from wandb.keras import WandbCallback
    wandb.init(project="SQUAD", name=wandb_experiment_name)
    wandb.config.batch_size = batch_size
    wandb.config.epochs = epochs
    

saveDir = os.path.join(os.getcwd(), 'saved_models')
if not os.path.isdir(saveDir):
    os.makedirs(saveDir)
chkpt = saveDir + '/squad_check.hdf5'

es_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto')
cp_cb = tf.keras.callbacks.ModelCheckpoint(filepath = chkpt, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
if ENABLE_WANDB:
    callbacks = [es_cb, cp_cb, WandbCallback(log_batch_frequency=10)]
else:
    callbacks = [es_cb, cp_cb]

model = build_model()
tf.keras.backend.clear_session()
model.compile(optimizer='adam', loss=custom_loss, metrics=[
                                                           precision_start, precision_end, 
                                                           mean_abs_dist_start, mean_abs_dist_end])

history = model.fit([np.stack(df_train['passage_pad']),np.stack(df_train['question_pad'])],np.stack(df_train['word_idx_answer'].to_numpy()), epochs=epochs,
                        callbacks=callbacks, validation_data=([np.stack(df_val['passage_pad']), np.stack(df_val['question_pad'])],np.stack(df_val['word_idx_answer'].to_numpy())),
                        batch_size=batch_size)

In [ ]:
#@title precedence plot {form-width: "20%"}

predictions = model.predict([np.stack(df_val['passage_pad']), np.stack(df_val['question_pad'])])
sampled_start = np.argmax(predictions[:,:,0], axis=-1)
sampled_end = np.argmax(predictions[:,:,1], axis=-1)
plt.figure(figsize=(30,30))
plt.plot(np.stack(df_val['word_idx_answer'])[:,0],np.stack(df_val['word_idx_answer'])[:,1], ".")
plt.plot(sampled_start, sampled_end,"*")

In [ ]:
print("end before start ratio")
precedence_violation = sum(sampled_end - sampled_start < 0) / len(sampled_end) * 100
print(precedence_violation)
if ENABLE_WANDB:
    wandb.log({"precedence violation": wandb.Html(
        "<pre>precedence violation: "+str(precedence_violation)+" %<pre>", inject=False)})

In [27]:
!pip install tokenizers

     |████████████████████████████████| 3.2MB 5.6MB/s 


In [28]:
import json
import os
import re
import string
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer

In [19]:
with open("error IDs.txt", "r") as f:
    unwanted_id = f.read()

unwanted_id = unwanted_id.split("\n")[:-1]

df_bert = df_original.set_index('id')

df_bert = df_bert.drop(unwanted_id)

In [ ]:
df_original

In [ ]:
def preprocess(self):
        # clean context and question
        context = " ".join(str(self.context).split())
        question = " ".join(str(self.question).split())
        # tokenize context and question
        tokenized_context = tokenizer.encode(context)
        tokenized_question = tokenizer.encode(question)
        # if this is validation or training sample, preprocess answer
        if self.answer_text is not None:
            answer = " ".join(str(self.answer_text).split())
            # check if end character index is in the context
            end_char_idx = self.start_char_idx + len(answer)
            if end_char_idx >= len(context):
                self.skip = True
                return
            # mark all the character indexes in context that are also in answer     
            is_char_in_ans = [0] * len(context)
            for idx in range(self.start_char_idx, end_char_idx):
                is_char_in_ans[idx] = 1
            ans_token_idx = []
            # find all the tokens that are in the answers
            for idx, (start, end) in enumerate(tokenized_context.offsets):
                if sum(is_char_in_ans[start:end]) > 0:
                    ans_token_idx.append(idx)
            if len(ans_token_idx) == 0:
                self.skip = True
                return
            # get start and end token indexes
            self.start_token_idx = ans_token_idx[0]
            self.end_token_idx = ans_token_idx[-1]
        # create inputs as usual
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(tokenized_question.ids[1:])
        attention_mask = [1] * len(input_ids)
        padding_length = max_seq_length - len(input_ids)
        # add padding if necessary
        if padding_length > 0:
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:
            self.skip = True
            return
        self.input_word_ids = input_ids
        self.input_type_ids = token_type_ids
        self.input_mask = attention_mask
        self.context_token_to_char = tokenized_context.offsets

In [ ]:
#BERT
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2", trainable=True)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy().decode("utf-8")

tokenizer = BertWordPieceTokenizer(vocab=vocab_file, lowercase=True)



def preprocess_bert(text):
    preprocesed_text = [" ".join(str(line).split()) for line in text]
    tokenized_text = [tokenizer.encode(preprocesed_line) for preprocesed_line in preprocesed_text]
    return tokenized_text

def labeling(df):
    answer = " ".join(str(df['answer_text']).split())
    
    # mark all the character indexes in context that are also in answer     
    is_char_in_ans = [0] * len(context)
    for idx in range(self.start_char_idx, end_char_idx):
        is_char_in_ans[idx] = 1
    ans_token_idx = []
    # find all the tokens that are in the answers
    for idx, (start, end) in enumerate(tokenized_context.offsets):
        if sum(is_char_in_ans[start:end]) > 0:
            ans_token_idx.append(idx)
    if len(ans_token_idx) == 0:
        self.skip = True
        return
    # get start and end token indexes
    self.start_token_idx = ans_token_idx[0]
    self.end_token_idx = ans_token_idx[-1]






df_bert_preprocessed = df_bert.copy()
# pre-process passage and question text
df_bert_preprocessed['passage'] = preprocess_bert(df_bert['passage'])
df_bert_preprocessed['question'] = preprocess_bert(df_bert['question'])

In [36]:
df_bert

,title,passage,question,answer_idx,answer_text
id,,,,,
5733be284776f41900661182,University_of_Notre_Dame,"Encoding(num_tokens=160, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","Encoding(num_tokens=17, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","(515, 541)","Encoding(num_tokens=10, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])"
5733be284776f4190066117f,University_of_Notre_Dame,"Encoding(num_tokens=160, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","Encoding(num_tokens=13, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","(188, 213)","Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])"
5733be284776f41900661180,University_of_Notre_Dame,"Encoding(num_tokens=160, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","Encoding(num_tokens=17, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","(279, 296)","Encoding(num_tokens=5, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])"
5733be284776f41900661181,University_of_Notre_Dame,"Encoding(num_tokens=160, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","Encoding(num_tokens=11, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","(381, 420)","Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])"
5733be284776f4190066117e,University_of_Notre_Dame,"Encoding(num_tokens=160, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","Encoding(num_tokens=14, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","(92, 126)","Encoding(num_tokens=9, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])"
...,...,...,...,...,...
5735d259012e2f140011a09d,Kathmandu,"Encoding(num_tokens=153, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","Encoding(num_tokens=14, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","(229, 235)","Encoding(num_tokens=3, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])"
5735d259012e2f140011a09e,Kathmandu,"Encoding(num_tokens=153, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","Encoding(num_tokens=10, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","(414, 421)","Encoding(num_tokens=4, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])"
5735d259012e2f140011a09f,Kathmandu,"Encoding(num_tokens=153, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","Encoding(num_tokens=14, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])","(476, 481)","Encoding(num_tokens=3, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])"
